# Capitulo 7 - Decoradores de função e closures

## Básico sobre decoradores

Um decorador é um invocável (callable) que aceita outra função como argumento (a função decorada).

O decorador pode realizar algum processamento com a função decorada e devolvê-la ou substituí-la por outra função ou um objeto invocável. Em outras palavras, supondo que haja um decorador chamado decorate, este código:

In [ ]:
@decorate()
def target():
  print('Running target()')

# Tem o mesmo efeito que escrever:

def target():
  print('running target()')

target = decorate(target)

In [8]:
# Um decorador normalmente substitui uma função por outra função diferente

def deco(func):
  def inner():
    print('running inner()')
    
  return inner

@deco
def target():
  print('running target()')

target()
target

running inner()


<function __main__.deco.<locals>.inner()>

## Quando Python executa os decoradores

Uma característica fundamental dos decoradores é que eles são executados imediatamente após a função decorada ser definida. Normalmente, isso ocorre em tempo de importação.

In [14]:
# Módulo registration.py

registry = []

def register(func):
  print('running register(%s)' % func)
  registry.append(func)
  return func

@register
def f1():
  print('running f1()')

@register
def f2():
  print('running f2()')

def f3():
  print('running f3()')

def main():
  print('running main()')
  print('registry ->', registry)
  f1()
  f2()
  f3()

# if __name__ =='__ main__':
main()

running register(<function f1 at 0x7f3ad4367ee0>)
running register(<function f2 at 0x7f3ad4348310>)
running main()
registry -> [<function f1 at 0x7f3ad4367ee0>, <function f2 at 0x7f3ad4348310>]
running f1()
running f2()
running f3()


## Padrão Strategy melhorado com decorador

In [16]:
# A lista promos é preenchida pelo decorador promotion

promos = []

def promotion(promo_func):
  promos.append(promo_func)
  return promo_func

@promotion
def fidelity (order):
  """5% de desconto para clientes con mil ou mais pontos no programa de fidelidade"""
  return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
  """10% de desconto para cada LineItem com 20 ou mais unidades"""
  discount = 0
  for item in order.cart:
    if item.quantity >= 20:
      discount += item.total() * .1
  return discount

@promotion
def large_order(order):
  """7% de desconto para pedidos com 10 ou mais itens diferentes"""
  distinct_items = {item.product for item in order.cart}
  if len(distinct_items) >= 10:
    return order.total() * .07
  return 0

def best_promo(order):
  """Seleciona o melhor desconto disponível"""
  return max(promo(order) for promo in promos)

## Regras para escopo de variáveis

In [18]:
# Função que lê uma variável local e uma variável global

b=6

def f1(a):
  print(a)
  print(b)

f1(3)

3
6


In [20]:
# Variável b é local porque ela recebe um valor no corpo da função

b = 6

def f2(a):
  print(a)
  print(b)
  b = 9

f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

In [27]:
# Se quisermos que o interpretador trate b como uma variável global apesar da atribuição dentro da função, devemos usar a declaração global:

def f3(a):
  global b
  print(a)
  print(b)
  b = 9

f3(3)
b
a = 3
b = 8
b = 30
b

3
30


30

## Closures

In [29]:
# average_oo.py: uma classe para calcular uma média em evolução

class Averager():
  def __init__(self):
    self.series = []
  
  def __call__(self, new_value):
    self.series.append(new_value)
    total = sum(self.series)
    return total/len(self.series)

In [32]:
avg = Averager()
avg(10)
avg(11)
avg(12)

11.0

In [33]:
# average.py: uma função de ordem superior para calcular uma média em evolução

def make_averager():
  series = []
  
  def averager(new_value):
    series.append(new_value)
    total = sum(series)
    return total/len(series)
  
  return averager

In [36]:
avg = make_averager()
avg(10)
avg(11)
avg(12)

11.0

In [42]:
# Inspecionando a função criada por make_averager

avg.__code__.co_varnames
avg.__code__.co_freevars
avg.__closure__
avg.__closure__[0].cell_contents

[10, 11, 12]

Para resumir: uma closure é uma função que preserva as associações com as variáveis livres existentes quando a função é definida, de modo que elas possam ser usadas posteriormente quando a função for chamada e o escopo de definição não estiver mais disponível.

## Declaração nonlocal

In [43]:
# Uma função de ordem superior com falha para tentar calcular uma média em evolução sem manter todo o histórico

def make_averager():
  count = 0
  total = 0
  
  def averager(new_value):
    count += 1
    total += new_value
    return total / count
  
  return averager

In [44]:
avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

In [45]:
# Calcula uma média em evolução sem manter todo o histórico (fixo com o uso de nonlocal)

def make_averager():
  count = 0
  total = 0
  
  def averager(new_value):
    nonlocal count, total
    count += 1
    total += new_value
    return total / count
  
  return averager

## Implementando um decorador simples

In [49]:
# Um decorador simples para apresentar o tempo de execução das funções

import time

def clock(func):
  def clocked(*args):
    t0 = time.perf_counter()
    result = func(*args)
    elapsed = time.perf_counter() - t0
    name = func.__name__
    arg_str = ''.join(repr(arg) for arg in args)
    print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))

    return result
  
  return clocked

In [50]:
# Usando o decorador clock

import time

@clock
def snooze(seconds):
  time.sleep(seconds)

@clock
def factorial(n):
  return 1 if n < 2 else n * factorial(n-1)

# if __name__ == '__ main__':
print('*' * 40, 'Calling snooze(.123)')
snooze(.123)
print('*' * 40, 'Calling factorial(6))')
print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12316204s] snooze(0.123) -> None
**************************************** Calling factorial(6))
[0.00000112s] factorial(1) -> 1
[0.00002996s] factorial(2) -> 2
[0.00004562s] factorial(3) -> 6
[0.00005894s] factorial(4) -> 24
[0.00007277s] factorial(5) -> 120
[0.00008873s] factorial(6) -> 720
6! = 720


In [51]:
# Um decorador dock melhorado

import time
import functools

def clock(func):
  
  @functools.wraps(func)
  def clocked(*args, **kwargs):
    t0 = time.time()
    result = func(*args, **kwargs)
    elapsed = time.time() - t0
    name = func.__name__
    arg_lst = []
    
    if args:
      arg_lst.append(', '.join(repr(arg) for arg in args))
    
    if kwargs:
      pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
      arg_lst.append(', '.join(pairs))
    
    arg_str = ', '.join(arg_lst)
    print('[%0.8fs] %s(%s) -> %r ' % (elapsed, name, arg_str, result))
    
    return result

  return clocked

## Decoradores da biblioteca-padrão

### Memoização com functools.lru_cache

In [52]:
# A maneira recursiva bastante custosa de calcular o enésimo número da série de Fibonacci

@clock
def fibonacci(n):
  if n < 2:
    return n
  return fibonacci(n-2) + fibonacci(n-1)

# if __name__=='__main__':
print(fibonacci(6))

[0.00000095s] fibonacci(0) -> 0 
[0.00000024s] fibonacci(1) -> 1 
[0.00008368s] fibonacci(2) -> 1 
[0.00000024s] fibonacci(1) -> 1 
[0.00000024s] fibonacci(0) -> 0 
[0.00000024s] fibonacci(1) -> 1 
[0.00002551s] fibonacci(2) -> 1 
[0.00003362s] fibonacci(3) -> 2 
[0.00012565s] fibonacci(4) -> 3 
[0.00000000s] fibonacci(1) -> 1 
[0.00000048s] fibonacci(0) -> 0 
[0.00000000s] fibonacci(1) -> 1 
[0.00000763s] fibonacci(2) -> 1 
[0.00001550s] fibonacci(3) -> 2 
[0.00000000s] fibonacci(0) -> 0 
[0.00000024s] fibonacci(1) -> 1 
[0.00000763s] fibonacci(2) -> 1 
[0.00000024s] fibonacci(1) -> 1 
[0.00000048s] fibonacci(0) -> 0 
[0.00000000s] fibonacci(1) -> 1 
[0.00000787s] fibonacci(2) -> 1 
[0.00001526s] fibonacci(3) -> 2 
[0.00003004s] fibonacci(4) -> 3 
[0.00005293s] fibonacci(5) -> 5 
[0.00018716s] fibonacci(6) -> 8 
8


In [56]:
# Implementação mais rápida usando caching

import functools

@functools.lru_cache()
@clock
def fibonacci(n):
  if n < 2:
    return n
  return fibonacci(n-2) + fibonacci(n-1)

# if __name__=='__main__':
print(fibonacci(6))

[0.00000095s] fibonacci(0) -> 0 
[0.00000048s] fibonacci(1) -> 1 
[0.00006199s] fibonacci(2) -> 1 
[0.00000048s] fibonacci(3) -> 2 
[0.00007415s] fibonacci(4) -> 3 
[0.00000024s] fibonacci(5) -> 5 
[0.00008559s] fibonacci(6) -> 8 
8


### Funções genéricas com dispatch simples

In [59]:
import html

def htmlize(obj):
  content = html.escape(repr(obj))
  
  return '<pre>{}</pre>'.format(content)

In [67]:
# htmlize gera HTML personalizado conforme os diferentes tipos de objeto

htmlize({1, 2, 3})
htmlize(abs)
htmlize('Heimlich & Co.\n- a game')
htmlize(42)
print(htmlize(['alpha', 66, {3, 2, 1}]))

<pre>[&#x27;alpha&#x27;, 66, {1, 2, 3}]</pre>


In [ ]:
# singledispatch cria um htmlize.register personalizado para reunir várias funções em uma função genérica

from functools import singledispatch
from collections import abc
import numbers
import html

@singledispatch
def htmlize(obj):
  content = html.escape(repr(obj))
  return '<pre>{}</pre>'.format(content)

@htmlize.register(str)
def _(text):
  content = html.escape(text).replace('\n', '<br>\n')
  return '<p>{0}</p>'.format(content)

@htmlize.register(numbers.Integral)
def _(n):
  return '<pre>{0} (0x{0:x})</pre>'.format(n)

@htmlize.register(tuple)
@htmlize.register(abc.MutableSequence)
def _(seq):
  inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
  return '<ul>\n<li>' + inner + '</li>\n</ul>'

## Decoradores empilhados

Quando dois decoradores `@d1` e `@d2` são aplicados a uma função `f` nessa ordem, o resultado será o mesmo que `f = d1(d2(f))`.

Em outras palavras, isto:

```py
@d1
@d2
def f():
  print('f')
```

é o mesmo que:

```py
def f():
  print('f')

f = d1(d2(f))
```

## Decoradores parametrizados

In [68]:
# Módulo registration.py do exemplo 7.2 condensado, repetido aqui por conveniência

registry = []

def register(func):
  print('running register(%s)' % func)
  registry.append(func)
  return func

@register
def f1():
  print('running f1()')
  print('running main()')
  print('registry ->', registry)

f1()

running register(<function f1 at 0x7f3ad4367790>)
running f1()
running main()
registry -> [<function f1 at 0x7f3ad4367790>]


### Um decorador de registro parametrizado

In [74]:
# Para aceitar parâmetros, o novo decorador register deve ser chamado como uma função

registry = set()

def register(active=True):
  def decorate(func):
    print('running register(active=%s)->decorate(%s)' % (active, func))
    
    if active:
      registry.add(func)
    else:
      registry.discard(func)
    
    return func
  return decorate

@register(active=False)
def f1():
  print('running f1()')

@register()
def f2():
  print('running f2()')

def f3():
  print('running f3()')

registry

running register(active=False)->decorate(<function f1 at 0x7f3ad4348040>)
running register(active=True)->decorate(<function f2 at 0x7f3abeef8310>)


{<function __main__.f2()>}

In [82]:
# Registrando uma função
register()(f2)

registry

# Removendo uma função do registry
register(active=False)(f2)

running register(active=True)->decorate(<function f2 at 0x7f3abeef8310>)
running register(active=False)->decorate(<function f2 at 0x7f3abeef8310>)


<function __main__.f2()>

### Decorador clock parametrizado

In [83]:
# Módulo clockdeco_param.py: o decorador dock parametrizado

import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_FMT):
  def decorate(func):
    def clocked(*_args):
      t0 = time.time()
      _result = func(*_args)
      elapsed = time.time() - t0
      name = func.__name__
      args = ', '.join(repr(arg) for arg in _args)
      result = repr(_result)
      print(fmt.format(**locals()))
      
      return _result
    return clocked
  return decorate

if __name__ == '__main__':
  @clock()
  def snooze(seconds):
    time.sleep(seconds)
  
  for i in range(3):
    snooze(.123)

[0.12321472s] snooze(0.123) -> None
[0.12317705s] snooze(0.123) -> None
[0.12315798s] snooze(0.123) -> None


In [84]:
# dockdeco_param_demo1.py

import time

@clock('{name}: {elapsed}s')
def snooze(seconds):
  time.sleep(seconds)
  
for i in range(3):
  snooze(.123)

snooze: 0.12314295768737793s
snooze: 0.12324404716491699s
snooze: 0.12318253517150879s


In [85]:
# clockdeco_param_demo2.py

import time


@clock('{name}({args}) dt={elapsed:0.3f}s')
def snooze(seconds):
  time.sleep(seconds)

for i in range(3):
  snooze(.123)

snooze(0.123) dt=0.123s
snooze(0.123) dt=0.123s
snooze(0.123) dt=0.123s
